In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [2]:
movies=pd.read_csv("movies.csv")

In [3]:
movies.head()

index     budget                                    genres   
0      0  237000000  Action Adventure Fantasy Science Fiction  \
1      1  300000000                  Adventure Fantasy Action   
2      2  245000000                    Action Adventure Crime   
3      3  250000000               Action Crime Drama Thriller   
4      4  260000000          Action Adventure Science Fiction   

                                       homepage      id   
0                   http://www.avatarmovie.com/   19995  \
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language   
0  culture clash future space war space colony so...                en  \
1  ocean drug abuse exotic island east india trad...                en   
2         spy based on novel secret agent sequel mi6                en   
3  dc comics crime fighter terrorist secret ident...                en   
4  based on novel mars medallion space travel pri...                en   

                             original_title   
0                                    Avatar  \
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  ... runtime   
0  In the 22nd century, a paraplegic Marine is di...  150.437577  ...   162.0  \
1  Captain Barbossa, long believed to be dead, ha...  139.082615  ...   169.0   
2  A cryptic message from Bond’s past sends him o...  107.376788  ...   148.0   
3  Following the death of District Attorney Harve...  112.312950  ...   165.0   
4  John Carter is a war-weary, former military ca...   43.926995  ...   132.0   

                                    spoken_languages    status   
0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released  \
1           [{"iso_639_1": "en", "name": "English"}]  Released   
2  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   
3           [{"iso_639_1": "en", "name": "English"}]  Released   
4           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline   
0                     Enter the World of Pandora.  \
1  At the end of the world, the adventure begins.   
2                           A Plan No One Escapes   
3                                 The Legend Ends   
4            Lost in our world, found in another.   

                                      title vote_average vote_count   
0                                    Avatar          7.2      11800  \
1  Pirates of the Caribbean: At World's End          6.9       4500   
2                                   Spectre          6.3       4466   
3                     The Dark Knight Rises          7.6       9106   
4                               John Carter          6.1       2124   

                                                cast   
0  Sam Worthington Zoe Saldana Sigourney Weaver S...  \
1  Johnny Depp Orlando Bloom Keira Knightley Stel...   
2  Daniel Craig Christoph Waltz L\u00e9a Seydoux ...   
3  Christian Bale Michael Caine Gary Oldman Anne ...   
4  Taylor Kitsch Lynn Collins Samantha Morton Wil...   

                                                crew           director  
0  [{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...      James Cameron  
1  [{'name': 'Dariusz Wolski', 'gender': 2, 'depa...     Gore Verbinski  
2  [{'name': 'Thomas Newman', 'gender': 2, 'depar...         Sam Mendes  
3  [{'name': 'Hans Zimmer', 'gender': 2, 'departm...  Christopher Nolan  
4  [{'name': 'Andrew Stanton', 'gender': 2, 'depa...     Andrew Stanton  

[5 rows x 24 columns]

In [4]:
movies.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
movies.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [7]:
movies.describe()

index        budget             id   popularity       revenue   
count  4803.000000  4.803000e+03    4803.000000  4803.000000  4.803000e+03  \
mean   2401.000000  2.904504e+07   57165.484281    21.492301  8.226064e+07   
std    1386.651002  4.072239e+07   88694.614033    31.816650  1.628571e+08   
min       0.000000  0.000000e+00       5.000000     0.000000  0.000000e+00   
25%    1200.500000  7.900000e+05    9014.500000     4.668070  0.000000e+00   
50%    2401.000000  1.500000e+07   14629.000000    12.921594  1.917000e+07   
75%    3601.500000  4.000000e+07   58610.500000    28.313505  9.291719e+07   
max    4802.000000  3.800000e+08  459488.000000   875.581305  2.787965e+09   

           runtime  vote_average    vote_count  
count  4801.000000   4803.000000   4803.000000  
mean    106.875859      6.092172    690.217989  
std      22.611935      1.194612   1234.585891  
min       0.000000      0.000000      0.000000  
25%      94.000000      5.600000     54.000000  
50%     103.000000      6.200000    235.000000  
75%     118.000000      6.800000    737.000000  
max     338.000000     10.000000  13752.000000

In [8]:
movies.shape

(4803, 24)

In [9]:
selected_features=['genres','keywords','tagline','cast','director']

In [10]:
for feature in selected_features:
    movies[feature]=movies[feature].fillna("")

In [11]:
combined_feature=movies['genres']+" "+movies['keywords']+" "+movies['tagline']+" "+movies['cast']+" "+movies['director']

In [12]:
print(combined_feature)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [13]:
vectorizer=TfidfVectorizer()

In [14]:
feature_vector=vectorizer.fit_transform(combined_feature)

In [15]:
print(feature_vector)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [16]:
similarity=cosine_similarity(feature_vector)

In [17]:
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [18]:
similarity.shape

(4803, 4803)

In [20]:
movie_name=input("enter movie name")
print(movie_name)

enter movie nameiron man
iron man


In [21]:
list_movies=movies['title'].tolist()

In [22]:
list_movies

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [23]:
matching=difflib.get_close_matches(movie_name,list_movies)

In [24]:
matching

['Iron Man', 'Iron Man 3', 'Iron Man 2']

In [25]:
matching[0]

'Iron Man'

In [26]:
index_of_movie=movies[movies.title==matching[0]]['index'].values[0]
index_of_movie

68

In [27]:
similarity_score=list(enumerate(similarity[index_of_movie]))

In [28]:
similarity_score

[(0, 0.033570748780675445),
 (1, 0.0546448279236134),
 (2, 0.013735500604224323),
 (3, 0.006468756104392058),
 (4, 0.03268943310073386),
 (5, 0.013907256685755473),
 (6, 0.07692837576335507),
 (7, 0.23944423963486405),
 (8, 0.007882387851851008),
 (9, 0.07599206098164225),
 (10, 0.07536074882460438),
 (11, 0.01192606921174529),
 (12, 0.013707618139948929),
 (13, 0.012376074925089967),
 (14, 0.09657127116284188),
 (15, 0.007286271383816743),
 (16, 0.22704403782296803),
 (17, 0.013112928084103857),
 (18, 0.04140526820609594),
 (19, 0.07883282546834255),
 (20, 0.07981173664799915),
 (21, 0.011266873271064948),
 (22, 0.006892575895462364),
 (23, 0.006599097891242659),
 (24, 0.012665208122549737),
 (25, 0.0),
 (26, 0.21566241096831154),
 (27, 0.030581282093826635),
 (28, 0.061074402219665376),
 (29, 0.014046184258938898),
 (30, 0.0807734659476981),
 (31, 0.31467052449477506),
 (32, 0.02878209913426701),
 (33, 0.13089810941050173),
 (34, 0.0),
 (35, 0.035350090674865595),
 (36, 0.03185325269

In [29]:
len(similarity_score)

4803

In [30]:
sorted_score=sorted(similarity_score,key=lambda x:x[1],reverse=True)

In [31]:
sorted_score

[(68, 1.0000000000000002),
 (79, 0.40890433998005965),
 (31, 0.31467052449477506),
 (7, 0.23944423963486405),
 (16, 0.22704403782296803),
 (26, 0.21566241096831154),
 (85, 0.20615862984665329),
 (182, 0.19573956139611606),
 (511, 0.16702973947860686),
 (3623, 0.1609246088135586),
 (64, 0.15299924139445145),
 (203, 0.14818667948665118),
 (174, 0.1471993120942043),
 (4401, 0.14505971470107848),
 (101, 0.14401677581826294),
 (46, 0.14216268867232237),
 (169, 0.1380947013224906),
 (1740, 0.13624382641690763),
 (94, 0.1361681957902901),
 (788, 0.1330589507422922),
 (126, 0.13263982780511066),
 (131, 0.13137698586006535),
 (33, 0.13089810941050173),
 (2487, 0.12309731939910507),
 (783, 0.12162995562040377),
 (138, 0.11846458075866884),
 (2442, 0.11725512335483321),
 (661, 0.11719294096248463),
 (607, 0.11387063493435637),
 (38, 0.1126182690487113),
 (2651, 0.1121878787373205),
 (353, 0.1116846512704428),
 (122, 0.10850296033661253),
 (1553, 0.1079782217151326),
 (1451, 0.107849394974707),
 (

In [32]:
print("Suggested Movies\n")
i=1
for movie in sorted_score:
    index=movie[0]
    title_index=movies[movies.index==index]['title'].values[0]
    if(i<20):
        print(title_index)
        i+=1

Suggested Movies

Iron Man
Iron Man 2
Iron Man 3
Avengers: Age of Ultron
The Avengers
Captain America: Civil War
Captain America: The Winter Soldier
Ant-Man
X-Men
Made
X-Men: Apocalypse
X2
The Incredible Hulk
The Helix... Loaded
X-Men: First Class
X-Men: Days of Future Past
Captain America: The First Avenger
Kick-Ass 2
Guardians of the Galaxy


In [33]:
#suggested movie function
movie_name=input("Enter the movie name::")

list_of_movies=movies["original_title"].tolist()

find_close_movies=difflib.get_close_matches(movie_name,list_of_movies)

close_movie=find_close_movies[0]

index_of_the_movie=movies[movies.original_title==close_movie]['index'].values[0]

similarity_score=list(enumerate(similarity[index_of_the_movie]))

sort_score=sorted(similarity_score,key=lambda x:x[1],reverse=True)

i=0;
for k in sort_score:
    index=k[0]
    title_of_the_movie=movies[movies.index==index]['title'].values[0]
    if(i<15):
        print(title_of_the_movie)
        i+=1

Enter the movie name::iron man
Iron Man
Iron Man 2
Iron Man 3
Avengers: Age of Ultron
The Avengers
Captain America: Civil War
Captain America: The Winter Soldier
Ant-Man
X-Men
Made
X-Men: Apocalypse
X2
The Incredible Hulk
The Helix... Loaded
X-Men: First Class
